# HUGGINGFACE PAGE:
https://huggingface.co/human-centered-summarization/financial-summarization-pegasus

# PREREQUISITES

In [1]:
#!pip install transformers datasets evaluate

In [2]:
#!pip install -U ipywidgets>=8

In [3]:
#pip install sentencepiece

# PEGASUS

In [4]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

In [5]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name) # If you want to use the Tensorflow model 
                                                                    # just replace with TFPegasusForConditionalGeneration

In [15]:
TEXT_0 = "National Commercial Bank (NCB), Saudi Arabia’s largest lender by assets, agreed to buy rival Samba Financial Group for $15 billion in the biggest banking takeover this year.NCB will pay 28.45 riyals ($7.58) for each Samba share, according to a statement on Sunday, valuing it at about 55.7 billion riyals. NCB will offer 0.739 new shares for each Samba share, at the lower end of the 0.736-0.787 ratio the banks set when they signed an initial framework agreement in June.The offer is a 3.5% premium to Samba’s Oct. 8 closing price of 27.50 riyals and about 24% higher than the level the shares traded at before the talks were made public. Bloomberg News first reported the merger discussions.The new bank will have total assets of more than $220 billion, creating the Gulf region’s third-largest lender. The entity’s $46 billion market capitalization nearly matches that of Qatar National Bank QPSC, which is still the Middle East’s biggest lender with about $268 billion of assets."

In [20]:
input_ids_0 = input_ids = tokenizer(TEXT_0, return_tensors="pt").input_ids

In [17]:
%%time
# Generate the output (Here, we use beam search but you can also use any other strategy you like)
output_ids_0 = model.generate(
    input_ids_0, 
    max_length=64, 
    num_beams=5, 
    early_stopping=True
)

CPU times: user 7min 11s, sys: 11.1 s, total: 7min 22s
Wall time: 15.1 s


In [18]:
SUMMARY_0 = tokenizer.decode(output_ids_0[0], skip_special_tokens=True)

In [14]:
# Finally, we can print the generated summary
print(SUMMARY_0)
# Generated Output: Saudi bank to pay a 3.5% premium to Samba share price. Gulf region’s third-largest lender will have total assets of $220 billion

Saudi bank to pay a 3.5% premium to Samba share price. Gulf region’s third-largest lender will have total assets of $220 billion


In [19]:
TEXT_1 = """
Alaska Governor Mike Dunleavy issued the following statement today regarding the unidentified object shot down this morning over Alaskan waters.

“Today’s announcement by the Pentagon that an unidentified object was shot down by an F-22 from Joint Base Elmendorf Richardson over Alaska’s coastline raises serious national security concerns that should concern every American. I want to thank our Alaska-based military for eliminating the potential threat before it was flying over our land.

This latest intrusion into our airspace raises serious questions about the White House’s decision to not shoot down a Chinese spy balloon last week when it was above the Aleutian Chain and prevent it from flying over important military sites in the Lower-48.

Unlike other states, Alaska is truly on the front lines. Because of our close proximity to our neighbors there is very little margin for error. Russian territory is only a few miles away. We are the one state closest to the Korean Peninsula and China. Alaska is truly on the front lines.

As such, decisions need to be made quickly to preserve the territorial integrity of Alaska and the United States. This incident is further evidence that the military capability of Alaska is robust. An important discussion should ensue about improving those capabilities, including icebreakers, Army, Air Force and Navy capabilities.

If the last few days are any indication, this may be the new norm and we must be prepared. The Alaska National Guard is working closely with USNORTHCOM and other agencies to provide any support as requested.

This latest incident demonstrates that Alaska remains the most strategic place on earth for both geopolitics and national defense.”
"""

In [21]:
input_ids_1 = input_ids = tokenizer(TEXT_1, return_tensors="pt").input_ids

In [22]:
%%time
# Generate the output (Here, we use beam search but you can also use any other strategy you like)
output_ids_1 = model.generate(
    input_ids_1, 
    max_length=64, 
    num_beams=5, 
    early_stopping=True
)

CPU times: user 1min 10s, sys: 603 ms, total: 1min 11s
Wall time: 2.33 s


In [23]:
SUMMARY_1 = tokenizer.decode(output_ids_1[0], skip_special_tokens=True)

In [24]:
print(SUMMARY_1)

F-22 shot down an unidentified object over Alaska’s coastline. Dunleavy says shooting down object removes potential threat to land


# ENCAPSULATE 

In [58]:
def summarize(text, *, max_output_len=64, max_input_len=512, chunk_len=512, num_beams=5, early_stopping=True, truncate=True, by_chunk=False):
    def _summarize(txt):
        if truncate:
            txt_ = txt[:max_input_len]
        else:
            txt_ = txt
        _input_ids = tokenizer(txt_, return_tensors="pt").input_ids
        _output_ids = model.generate(_input_ids, 
                                    max_length=max_output_len, 
                                    num_beams=num_beams, 
                                    early_stopping=early_stopping
                                    )
        _summary = tokenizer.decode(_output_ids[0], skip_special_tokens=True)
        return _summary
    if by_chunk:
        text_len = len(text)
        summaries = []
        for hi in range(0, text_len, chunk_len):
            lo = hi
            hi = lo + chunk_len
            chunk = text[lo:hi]
            chunk_summary = _summarize(chunk)
            summaries.append(chunk_summary)
    else:
        summaries = [_summarize(text)]
    return summaries

In [59]:
TEXT_2 = """
Toronto Mayor John Tory abruptly resigned on Friday after saying he had a relationship with a staff member, only months after cruising to reelection on a pledge to fix a housing crisis in North America’s fourth-largest city.

“During the pandemic, I developed a relationship with an employee in my office in a way that did not meet the standards to which I hold myself as mayor and as a family man,” Tory said at a press conference. “I’ve decided that I would step down as mayor so that I could take the time to reflect on my mistakes and to do the work of rebuilding the trust of my family.”

Canada’s financial capital is facing a number of challenges, including the lingering financial effects of Covid-19 lockdowns, a potential recession and a housing affordability crisis. Tory’s promise to fix the latter was one of his main campaign pledges when he won an unprecedented third consecutive term in October. 

Canada saw some of the fastest house price inflation in the world, while Toronto was named the world’s biggest housing bubble last year by Swiss bank UBS Group. The city has not been building homes fast enough to keep up with immigration-led population growth and a lack of supply is seen as the main cause of soaring prices that, while off their peak, remain lofty.

Tory had proposed a major policy shift that would open residential land reserved for single-family homes for the development of multiplexes and small apartment buildings, with mid-rises on commercial streets. 

During Friday’s news conference, Tory said he began a relationship with an employee in his office during the pandemic. The staffer left City Hall to work elsewhere during the affair and the relationship ended “by mutual consent” earlier this year, he said. He took no questions.  

“I think it is important, as I always have, for the office of the mayor not to be in any way tarnished and not to see the city government itself put through a period of prolonged controversy, arising out of this error in judgment on my part, especially in light of some of the challenges that we face as a city,” said Tory, who was elected to the office in 2014.

Even-featured and gray-haired, Tory, 68, has long been a public figure in Toronto — a former leader of a provincial political party, business executive and talk radio host. He hails from an established Toronto business family, whose name adorns a storied law firm, Torys LLP, founded by his grandfather. He has been married for more than 40 years.

Tory said he will work with the city manager, city clerk and Deputy Mayor Jennifer McKelvie to ensure an orderly transition. An environmental geoscientist, McKelvie was first elected as a city councilor in 2018. Under Ontario legislation governing the city, it will hold a special election to fill the mayor’s office in the coming months.  


"""

In [107]:
%%time
SUMMARY_2 = summarize(TEXT_2, max_output_len=256)[0]
print(SUMMARY_2)

John Tory says he had a relationship with employee. He says he will not seek re-election in 2020
CPU times: user 3min 17s, sys: 150 ms, total: 3min 18s
Wall time: 6.73 s


In [61]:
%%time
SUMMARIES_2 = summarize(TEXT_2, by_chunk=True)


['John Tory says he had a relationship with employee. He says he will not seek re-election in 2020', 'Toronto Mayor John Tory says he’s taking a break from politics.', 'Toronto Mayor John Tory drops plan to open up land for development.', 'Toronto Mayor Rob Ford has admitted to having an affair.', 'John Tory has been re-elected as mayor of Canada’s largest city.', 'City Councilor McKelvie named interim mayor.']
CPU times: user 2h 6min 43s, sys: 10.5 s, total: 2h 6min 54s
Wall time: 4min 24s


In [62]:
[print(_) for _ in SUMMARIES_2]

John Tory says he had a relationship with employee. He says he will not seek re-election in 2020
Toronto Mayor John Tory says he’s taking a break from politics.
Toronto Mayor John Tory drops plan to open up land for development.
Toronto Mayor Rob Ford has admitted to having an affair.
John Tory has been re-elected as mayor of Canada’s largest city.
City Councilor McKelvie named interim mayor.


[None, None, None, None, None, None]

In [63]:
%%time
SUMMARY_2 = '\n'.join(SUMMARIES_2)
META_SUMMARY_2 = summarize(SUMMARY_2)[0]

CPU times: user 49 s, sys: 130 ms, total: 49.1 s
Wall time: 1.64 s


In [64]:
print(META_SUMMARY_2)

Toronto Mayor says he will not seek re-election in 2020. Rob Ford has admitted to having an affair


In [66]:
TEXT_3 = """
When news broke that Trafigura Group faces more than half a billion dollars in losses from what it described as a “systematic fraud,” the biggest surprise for many market insiders wasn’t the commodity trader’s missing nickel cargoes. It was that one of the industry’s largest players was still doing business with a man that others had long since backed away from.

Indian businessman Prateek Gupta and his companies, against whom Trafigura secured a $625 million freezing order this week, have a checkered history in the trading world.

Merchant Gunvor Group and trade finance fund TransAsia Private Capital Ltd. lost money in earlier dealings with Gupta’s companies, public filings show. Others, including banks and counterparties, became uncomfortable at times with the group’s trading activities, according to several people who either worked at the group or did business with it. Last year, India’s federal police announced it’s investigating allegations of fraud against Gupta himself.

Ian Milne, a former commodity trade finance executive at Rabobank and HSBC Holdings Plc who worked at TransAsia for two years in 2018 to 2020 trying to recover debts from Gupta’s companies, said he “had to rub my eyes a couple of times” when he saw the news this week.

“It’s very well known in the market that these guys have a highly dubious reputation,” Milne said in an interview. “Most people haven’t dealt with them for many years.”

Trafigura’s losses have shocked the commodity trading world, demonstrating that even one of the largest companies isn’t able to avoid the kind of blowups and risks that have plagued the industry in recent years, and raising questions about the whole sector’s risk management.

Bloomberg made multiple attempts  by phone and email to reach Gupta and companies owned by, or otherwise linked to him for comment on Friday but did not receive any response.

Trafigura said that it began investigating after identifying a number of red flags.

“This was a systematic fraud perpetrated after a long and legitimate business relationship dating back to 2015 that involved misrepresentation and widespread falsification of primary and supporting documentation,” a spokeswoman said. “Any fraud is an opportunity to review and tighten systems and procedures and a thorough review is underway.”

Gupta was born in 1979 into a commodity trading family. His father Vijay was a steel trader who represented Brazilian and Spanish companies importing iron and steel products into India in the 1980s and 1990s, according to a company obituary. When he died in 2009, Prateek took over the running of the family company, Mumbai-based Ushdev International Ltd., with his mother Suman as chair. At its peak in the early 2010s, the company had a market capitalization of about $250 million.

In person, Gupta is charming and rarely flustered, say several people who’ve done business with him. 

“He has a very laid-back style. Whatever the problem is, he’ll say it’s not really a problem, it’s all going to be solved,” said Milne, who now works for MonetaGo, which builds technology to help banks and others avoid trade-finance frauds.

Jonas Rey, chief executive officer of Athena Intelligence, a corporate intelligence company in Geneva that provides support to trade finance entities, said he investigated entities including TMT on behalf of several clients.

“We provided intel to multiple clients on TMT’s involvement in what we called a carousel fraud,” he said. “You have one cargo in the middle, you create 10 companies around it, and they sell the cargo to one another. One cargo gets financed 10 times. It’s like a financial musical chair. Eventually everything comes crashing down.”

Several companies came to regret their dealings with Gupta. Gunvor was left with exposure to Ushdev in the tens of millions of dollars when it got into financial trouble, according to company filings and people familiar with the matter. The exposure only led to a relatively small loss for Gunvor as it claimed on its insurance. Still, it was a blow that coincided with Gunvor’s decision to close its metal-trading desk in 2016.

A list of creditors published when Ushdev went into insolvency in 2018 showed that Gunvor was its largest non-bank creditor with an exposure of 3 billion rupees (about $45 million at the time).

A Gunvor spokesman declined to comment.

TransAsia is still embroiled in legal disputes with Gupta companies over alleged unpaid debts linked to its financing of trade in metals including copper. In one instance that’s been made public in Singapore courts, the trade finance fund claims that Gupta’s UD Trading Group Holding owes it $63 million. UD Trading has in the past said it doesn't believe it's liable for the debt.

Last July, the Central Bureau of Investigation – India’s equivalent of the FBI – said it had opened an investigation into Prateek and Suman Gupta, as well as Ushdev, over alleged fraud after a complaint made by State Bank of India. The CBI said in a press release it had conducted searches in three locations which had led to the “recovery of incriminating documents/articles.”

For Trafigura, the saga will raise difficult questions over how it vets its business partners.

“The postmortem will probably be ruthless internally,” said Jean-Francois Lambert, a consultant and former trade finance banker. “Traders and risk management will be challenged.”

"""

In [67]:
%%time
SUMMARIES_3 = summarize(TEXT_3, by_chunk=True)


CPU times: user 2h 21s, sys: 9.54 s, total: 2h 31s
Wall time: 4min 9s


In [69]:
for _ in SUMMARIES_3:
    print(_)

Indian businessman has been at the heart of fraud allegations. Trafigura’s losses could top half a billion dollars
Some banks and firms became uncomfortable with trading activities. Others, including banks and counterparties, became uncomfortable at times
Milne worked at TransAsia for two years trying to recover debts. Trafigura’s losses have shocked the commodity trading world
Trafigura says ‘ systematic fraud’ carried out by Gupta’s companies. Questions remain about risk management in oil industry
Prominent Indian commodities trader Sanjay Gupta has been suspended by his firm.
Prateek Gupta is one of India’s richest men, with a fortune of more than $1 billion.
TMT was involved in a ‘carousel fraud,’ Athena CEO says.
Glencore, Vitol also had exposure to Ushdev, people familiar say.
Ushdev’s exposure to Gunvor was about $45 million. TransAsia is still embroiled in legal disputes with Gupta companies
Prateek Gupta, Suman Gupta and Ushdev are being investigated. State Bank of India has a

In [70]:
%%time
SUMMARY_3 = summarize(TEXT_3)

CPU times: user 8min 12s, sys: 783 ms, total: 8min 13s
Wall time: 16.9 s


In [71]:
print(SUMMARY_3)

['Indian businessman has been at the heart of fraud allegations. Trafigura’s losses could top half a billion dollars']


In [73]:
%%time
META_SUMMARY_3 = summarize('\n'.join(SUMMARIES_3))

CPU times: user 11min 18s, sys: 796 ms, total: 11min 19s
Wall time: 23.3 s


In [74]:
print(META_SUMMARY_3)

['Milne worked at TransAsia for two years trying to recover debts. Trafigura says ‘ systematic fraud’ carried out by Gupta’s companies']


In [75]:
TEXT_4 = """
When the Montgomery County Police Department vacated its Third District station in 2014, it offered an opportunity to build something new right on the edge of downtown Silver Spring, Maryland — a suburb dealing with growing pains.

The unincorporated neighborhood, located just over the border with Washington, DC, has enjoyed a major urban revival following decades of decline. Part of this transformation has taken the form of a thriving arts and entertainment district, with a new civic building opened in 2010 and a shopping mall overhaul completed in 2015.


All this change made the 2.5-acre lot occupied by the old police station a hot ticket — and a point of contention. Home and rental prices in Montgomery County — and much of the DC area — have soared in recent years, but single-family homeowners have resisted new multifamily development. After county officials proposed building affordable housing on the lot, residents who lived next to the site pitched an alternative: an arts center to fill the dearth of working spaces for local artists, who they said were getting priced out of the area.

This disagreement could have led to a bitter land-use battle over 801 Sligo Avenue. Instead, the two ideas came together in a proposal from Artspace, a Minneapolis-based nonprofit developer that builds affordable live-work spaces for low-income artists. The group had previously completed two artist housing projects in the area: a 44-unit building along the historic corridor of suburban Mount Rainier, Maryland, and a 39-unit complex in DC’s Brookland neighborhood.

relates to In the DC Suburbs, an Artful Compromise Over Density and Housing
Photos: BKV Group. Illustration: Stephanie Davidson
Completed in 2020, Artspace Silver Spring is a mixed-used artist campus comprising a four-story apartment complex with a total of 68 affordable units and 11 for-sale townhomes wrapped around a central courtyard. Each apartment unit is restricted to applicants earning less than 60% of the area median income, with preference given to artists.

The first floor of the former police station — a white-brick structure built in the 1960s — has been preserved as the hub of the development and converted into studio spaces for lease, with glass entrances that open up to a paved public plaza.


For architects at BKV Group, a partner on the project, designing for artists was the easy part: Units are roughly 20% larger than comparable market-rate apartments, with open floor plans, high ceilings and wide entryways and corridors to accommodate those who work with larger-than-usual mediums. Communal areas like the main lobby and parts of the hallways double as gallery space for artists to display their work, and the outdoor spaces offer residents and the public a place to gather.

“It’s a contemporary interpretation of an old industrial artist loft, with materials meant to complement [those] of the police station,” says Willy Bermudez, senior design leader and partner at the firm.

relates to In the DC Suburbs, an Artful Compromise Over Density and Housing
Photos: BKV Group. Illustration: Stephanie Davidson
More challenging was creating something that seamlessly connects Silver Spring’s busy commercial district to the quiet neighborhood next door — and that met the demands of the site’s immediate neighbors, who were involved in nearly every step of the design process. “Zoning was a key element of this process because the site was between a heavily commercial area with tall buildings, and a single-family residential area,” says Bermudez. “We had to transition the project from one side to the other.”

The team decided to build the apartment complex on the sides that shared a street with the commercial area, to act as a screen between the two zones. Across the courtyard, the row of townhouses faces the existing neighborhood; their welcoming front porches and design borrow elements from the 1920s bungalow-style homes directly across the street.

“The [neighbors] wanted some of the design to feel like part of our neighborhood,” says Karen Roper, a community activist formerly with East Silver Spring Citizens Association, who led the charge on ensuring that residents’ voices were heard during the planning and construction process. “They didn’t want the campus to look like it was separate from the neighborhood. That was the big issue for them — and [the developers] listened to us very well.”

Each townhome has a garage that opens up to the courtyard instead of the street, while the one shared by apartment dwellers — located beneath the complex — was built without exterior walls to keep the campus from feeling closed off. To better manage stormwater runoff and reduce pollution from paved surfaces, bioswales scattered on the grounds help absorb excess rainfall. And instead of wrapping the facade of the apartment and townhomes with vibrant colors, BKV Group settled on a darker and more mellow scheme, adding pops of yellow accents.

relates to In the DC Suburbs, an Artful Compromise Over Density and Housing
Photos: BKV Group. Illustration: Stephanie Davidson
The project reflects the tricky balance between addressing an area’s severe housing shortage and getting community buy-in. Too much deference to public input stands to delay progress on adding new homes, or limit their ability to serve those in need. Opposition from existing homeowners can be powerful enough to cancel entire projects. Artist housing, too, can be a form of compromise over subsidized housing: A 2016 study from the University of Minnesota found that several such developments had far fewer non-white tenants than than other kinds of low-income housing in the Twin Cities. In its application process, Artspace emphasizes a commitment to attracting “individuals and families from diverse artistic and cultural backgrounds” — which shouldn’t be difficult, given Silver Spring’s ethnic diversity. 

As crowded metros like Montgomery County look to ramp up housing production and add apartment buildings to single-family neighborhoods in coming years, such tensions stand to amplify. Some homeowners are vehemently opposed to all growth; others just want more say in what gets built near them. As in this artist housing project, the design process may well be an area where developers, officials and residents can find common ground — ideally without losing sight of the long-term affordable housing goals.

Roper said she’d like to see more development efforts take a bottom-up approach that emphasizes working with the community. “It was win-win for everybody,” she says.
"""

In [76]:
%%time
SUMMARY_4 = summarize(TEXT_4)
# A former police station finds new life as low-income housing for artists in a development that bridges the divide between commercial and residential strips.

CPU times: user 8min 22s, sys: 259 ms, total: 8min 23s
Wall time: 17.2 s


In [77]:
print(SUMMARY_4)

['A new civic building is part of a thriving arts and entertainment district.']


In [79]:
%%time
SUMMARIES_4 = summarize(TEXT_4, by_chunk=True)

CPU times: user 1h 18min 51s, sys: 3.65 s, total: 1h 18min 54s
Wall time: 2min 42s


In [80]:
for _ in SUMMARIES_4:
    print(_)

A new civic building is part of a thriving arts and entertainment district.
A former police station site became the site of an arts center
Developer wants to build housing for low-income artists.
affordable housing complex in Washington, DC. Artspace Silver Spring is a mixed-used artist campus
Studio apartments in New York’s Meatpacking District are designed for artists.
Bermudez’s design for a police station in Washington, D.C.
Site was a prime location in the heart of Silver Spring, Maryland. BKV’s Bermudez worked closely with the city’s planning department.
Team wanted to create a ‘screen’ between commercial and residential areas
Residents say they were ignored during construction process
BKV Group designed a mixed-use development in Washington, DC, with a focus on sustainability.
Controversial artist housing can help solve housing crisis.
Artspace plans to build 200 apartments in Silver Spring. Some homeowners are vehemently opposed to all gro
Some want more affordable housing, oth

In [81]:
%%time
META_SUMMARY_4 = summarize('\n'.join(SUMMARIES_4))

CPU times: user 11min 56s, sys: 418 ms, total: 11min 56s
Wall time: 24.6 s


In [82]:
print(META_SUMMARY_4)

['Bermudez’s design for a police station in Washington, D.C. Site was a prime location in Silver Spring, Maryland.']


In [100]:
URL_5 = "https://www.bloomberg.com/news/features/2023-02-11/suburban-police-station-finds-new-life-as-artist-housing?srnd=premium&sref=dFV3EAbi"
TEXT_5 = """
Nikola Corp. has started work on hydrogen plants, part of a fueling network the company intends to underpin its push to compete in the nascent zero-emissions truck market. 

The seven-year-old manufacturer has also signed up a handful of hydrogen producers that have agreed to supply hydrogen for Nikola vehicles in parts of the U.S. and Canada, as the company seeks to have enough fuel for about 7,500 heavy-duty trucks by 2026.

The planned supply network is critical for Nikola’s strategy to begin assembling hydrogen fuel-cell electric trucks for the market later this year. Unlike battery-electric vehicles that can be recharged with electricity in many existing places, Nikola’s trucks need hydrogen to keep them on the road.

“The infrastructure for hydrogen doesn’t really exist for heavy transport,” said Carey Mendes, president of Nikola’s energy business. “If we’re going to sell hydrogen fuel-cell trucks, we have to have that infrastructure.”


Nikola, based in Arizona, said it began work late last year on a plant in the Phoenix suburb of Buckeye to extract hydrogen from water. Nikola forecast the initial hydrogen production volume at about 30 metric tons a day by late 2024. The company said it is negotiating an investment agreement for the plant with Fortescue Future Industries, a unit of Australian iron ore company Fortescue Metals Group Ltd.

A companion storage plant is planned at the site with hydrogen company Plug Power Inc., Nikola said. The storage plant will be able to turn hydrogen gas into a liquid at very low temperatures, making it easier to transport. Nikola said it intends to ship the hydrogen by truck to the company’s first three filling stations in California.

The hydrogen fuel network represents one of Nikola’s biggest challenges as it prepares to compete with battery-electric truck rivals, including electric-car maker Tesla Inc. and diesel truck manufacturers rolling out electric models. Many established truck makers have said they expect the limited availability of hydrogen to hold down demand for hydrogen trucks, and predict battery-electric trucks will become the first choice for truck operators seeking an electric alternative to diesel.  

Nikola’s executives are trying to repair the company’s reputation after founder and former chief executive Trevor Milton last year was found guilty of securities fraud for what prosecutors described as repeated lies to investors and others about Nikola’s trucks. The company said it has cooperated with government investigations and is focused on its business strategy.

Nikola settled a Securities and Exchange Commission probe for $125 million.

The company, which saw its stock price soar to nearly $80 a share shortly after going public in 2020, has leaned heavily on outside partners for hydrogen fuel plants. Nikola’s cash reserves have diminished and its stock price has dropped below $3, making it increasingly difficult to bankroll projects on its own, analysts said. 

Nikola said it has orders for about 1,000 hydrogen trucks, with 800 going to beer brewer Anheuser-Busch InBev NV. Nikola in the fall scaled back production of its own battery-electric model to conserve its cash. 

A chemical reaction between oxygen and hydrogen creates electricity that powers the truck’s electric motor. Nikola said its hydrogen trucks can travel about 500 miles before having to refuel with hydrogen, compared with a total of about 300 miles for some trucks with rechargeable batteries.


Nikola trucks will initially be offered on leases that the company said will include the cost of hydrogen fuel factored into lease payments.

Nikola said it has hydrogen supply commitments from Pennsylvania-based KeyState Natural Gas Synthesis LLC, Wabash Valley Resources LLC in Indiana, Canada’s TC Energy Corp. and New York-based Plug Power. To meet its trucks’ targeted zero-emissions profile, Nikola needs to obtain hydrogen from the limited supplies made from low-carbon or zero-emissions processes. 

Nikola is offering suppliers the prospect of a dedicated fleet of trucks that would consume their hydrogen. “If we can get hydrogen to our customers at a competitive rate, they’ll buy more trucks and we’ll buy more hydrogen,” Mr. Mendes said.

Nikola’s trucks initially will be offered on leases that will include the cost of hydrogen fuel, which Nikola said will be factored into lease payments at a fixed price or a variable rate.


KeyState said it would provide Nikola with up to 100 metric tons of hydrogen a day from natural gas wells in central Pennsylvania. Perry Babb, KeyState’s chief executive, said he expects his company to become Nikola’s primary hydrogen supplier for the U.S. mid-Atlantic region. 

“We’re willing to do a long-term agreement that gives us some security in an immature hydrogen market,” said Mr. Babb, who didn’t disclose the terms for KeyState’s supply agreement with Nikola.


Plug Power, which specializes in extracting the hydrogen from water, will start providing hydrogen to Nikola this year. Plug Power CEO Andy Marsh said he isn’t worried about ending up with too much production capacity for hydrogen if the actual demand for Nikola’s trucks turns out lower than anticipated.

“I have places to put this hydrogen regardless,” he said. “The risks are pretty low.”    

Nikola plans to locate its first three fueling stations near Los Angeles. The truck maker says it will brand its hydrogen and stations under the name “Hyla,” combining “hydrogen” and “Nikola.”

The company anticipates as many as 60 Hyla stations in the U.S. in three years, many in California where the state is trying to encourage the use of zero-emissions vehicles with financial incentives.

About 1,500 heavy-duty electric trucks were built in North America in 2022, about 0.5% of all the heavy-duty tractors produced, said truck-market forecaster ACT Research. Preston Feight, chief executive of Paccar Inc.—the maker of Peterbilt and Kenworth trucks—said he expects diesel engines to remain dominant in commercial trucks for several more years, but he is already selling battery-electric trucks and has hydrogen electric models under development.

“If one brings a distinct advantage to our customers, we’re ready to offer it to them,” he said during a January conference call with analysts. “We’ll put in the market what makes sense financially.” 
"""

In [112]:
%%time
SUMMARY_5 = summarize(TEXT_5, max_output_len=128, num_beams=10)

CPU times: user 3min 17s, sys: 189 ms, total: 3min 17s
Wall time: 6.73 s


In [113]:
print(SUMMARY_5)

['Fueling stations to be built in U.S., Canada. Truckmaker aims to have enough hydrogen for 7,500 vehicles']


In [103]:
%%time
SUMMARIES_5 = summarize(TEXT_5, by_chunk=True)

CPU times: user 1h 36min 8s, sys: 3.98 s, total: 1h 36min 12s
Wall time: 3min 18s


In [104]:
for _ in SUMMARIES_5:
    print(_)

Fueling stations to be built in U.S., Canada. Truckmaker aims to have enough hydrogen for 7,500 vehicles
Truckmaker is working on a network of hydrogen stations. Unlike battery-electric vehicles, Nikola’s trucks need hydrogen
Nikola plans to build a $1 billion water-based hydrogen plant.
Truck maker plans to start hydrogen-powered filling stations in California.
Fuel-cell truck maker says battery-electric vehicles will be ‘first choice’ for operators. CEO says company is ‘getting back on track’
Shares have dropped below $3 for the first time since going public
Anheuser-Busch’s Nikola to offer hydrogen-powered trucks.
Truck maker is offering hydrogen suppliers aDedicated fleet of vehicles.
KeyState says it will provide up to 100 tons of hydrogen a day to Nikola.
KeyState to start providing hydrogen to Nikola, CEO says. Plug Power to start providing hydrogen to Nikola this year
Truckmaker plans to open as many as 60 hydrogen stations in U.S.
More than half of new heavy-duty electric truc

In [105]:
%%time
META_SUMMARY_5 = summarize('\n'.join(SUMMARIES_5))

CPU times: user 55.5 s, sys: 39.3 ms, total: 55.6 s
Wall time: 1.85 s


In [106]:
print(META_SUMMARY_5)

['Truckmaker to build $1 billion water-based hydrogen plant. Unlike battery-electric vehicles, Nikola’s trucks need hydrogen']
